# BioData Catalyst Powered by PIC-SURE: Validate stigmatizing variables

The purpose of this notebook is to validate stigmatizing variables in [BioData Catalyst Powered by PIC-SURE](https://picsure.biodatacatalyst.nhlbi.nih.gov/). Specifically, this notebook will ensure the stigmatizing variables identified were removed from PIC-SURE Open Access.

For more information about stigmatizing variables, please view the [README.md](https://github.com/hms-dbmi/biodata_catalyst_stigmatizing_variables#biodata_catalyst_stigmatizing_variables).

### Install packages

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

In [ ]:
import PicSureClient
import PicSureBdcAdapter
from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol
from python_lib.stig_utils import validate_stig_vars
import pandas as pd

### Connect to PIC-SURE

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure" 
resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Be sure to use Open Access resource id
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)

### Get concept paths from PIC-SURE Open Access

To ensure that all stigmatizing variables were removed from PIC-SURE Open Access, we will compare the previously identified stigmatizing variables to a list of all variables in Open Access. 

In [ ]:
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)
dictionary = bdc.useDictionary().dictionary()

In [ ]:
variables = dictionary.find() # Input with study or list of studies that if needed
variables = variables.dataframe()

#variables

### Validation testing

`validate_stig_vars` is a function that compares the list of previously identified stigmatizing variables to the variables in PIC-SURE Open Access. If stigmatizing variables are found in Open Access, it will save the variables to a specified output file. 

| Function | Arguments / Input | Output|
|--------|-------------------|-------|
| `validate_stig_vars()` | (1) fullVariableDict of Open Access variables, (2) tab-delimited list of stigmatizing variables - output from identify_stigmatizing_variables.ipynb, (3) output file name | list of stigmatizing variables found in Open Access, if any |

In [ ]:
input_file = 'stigmatizing_variable_results/REVAMP_stigmatizing_variables.txt'
output_file = 'stigmatizing_variable_results/validation1.txt'

In [ ]:
results = validate_stig_vars(variables, input_file, output_file)